In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import reward as rw
import reward.utils as U

/home/quazar/anaconda3/envs/torchrl_up/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
GAMMA = 0.99

In [3]:
use_cuda = torch.cuda.is_available()
device   = torch.device("cuda" if use_cuda else "cpu")

In [4]:
env = rw.envs.GymEnv("InvertedPendulum-v2")
runner = rw.runners.SingleRunner(env)
batcher = rw.batchers.RolloutBatcher(runner=runner, batch_size=512)

Choosing the latest nvidia driver: /usr/lib/nvidia-390, among ['/usr/lib/nvidia-375', '/usr/lib/nvidia-390']
Choosing the latest nvidia driver: /usr/lib/nvidia-390, among ['/usr/lib/nvidia-375', '/usr/lib/nvidia-390']
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [5]:
class PolicyNN(nn.Module):
    def __init__(self, num_inputs, num_outputs, activation=nn.Tanh):
        super().__init__()
        self.activation = activation()
        
        self.hidden = nn.Linear(num_inputs, 64)
        self.mean = nn.Linear(64, num_outputs)
        self.log_std = nn.Parameter(torch.zeros(1, num_outputs))
        
    def forward(self, x):
        x = self.activation(self.hidden(x))
        mean = self.mean(x)
        log_std = self.log_std.expand_as(mean)
        return mean, log_std        

In [6]:
class ValueNN(nn.Module):
    def __init__(self, num_inputs, activation=nn.Tanh):
        super().__init__()
        self.activation = activation()
        
        self.hidden = nn.Linear(num_inputs, 64)
        self.out = nn.Linear(64, 1)
        
    def forward(self, x):
        x = self.activation(self.hidden(x))
        return self.out(x)

In [7]:
class GaussianPolicy(rw.policy.BasePolicy):
    def create_dist(self, state):
        mean, log_std = self.nn(state)
        return rw.distributions.Normal(loc=mean, scale=log_std.exp())
    
    def get_action(self, state, step):
        dist = self.create_dist(state)
        return U.to_np(dist.sample())

In [8]:
p_nn = PolicyNN(batcher.state_info.shape[0], batcher.action_info.shape[0]).to(device)
v_nn = ValueNN(batcher.state_info.shape[0]).to(device)
policy = GaussianPolicy(p_nn)

p_opt = torch.optim.Adam(p_nn.parameters(), lr=1e-3)
v_opt = torch.optim.Adam(v_nn.parameters(), lr=1e-3)

In [9]:
logger = U.Logger('logs/inverted_pendulum/v0-0')
last_logged_step = 0

Writing logs to: logs/inverted_pendulum/v0-0


In [10]:
for batch in batcher.get_batches(max_steps=5e5, get_action_fn=policy.get_action):
    batch = batch.to_tensor()
    # Calculate state value
    state_t = U.join_first_dims(batch.state_t, 2)
    v_t = v_nn(state_t)
    # Calculate return
    ret = U.estimators.discounted_sum_rewards(
        rewards=batch.reward,
        dones=batch.done,
        gamma=GAMMA,
        last_state_value_t=v_t[-1]
    ).detach()
    batch = batch.concat_batch()
    # Calculate advantage    
    adv = (ret - v_t).detach()
    
    # Calculate policy loss
    dist = policy.create_dist(batch.state_t)
    log_prob = dist.log_prob(batch.action)
    assert ret.shape == log_prob.shape
    p_loss = -(adv * log_prob).mean()
    
    # Calculate value loss
    v_loss = F.mse_loss(v_t, ret)
    
    # Optimize
    p_opt.zero_grad()
    p_loss.backward()
    p_opt.step()
    
    v_opt.zero_grad()
    v_loss.backward()
    v_opt.step()
    
    # Write logs    
    if batcher.num_steps > last_logged_step:
        last_logged_step = batcher.num_steps + 10000
        batcher.write_logs(logger)
        logger.add_log('policy/loss', p_loss)
        logger.add_log('v/loss', v_loss)
        logger.log(step=batcher.num_steps)      


                          Step 512                          
--------------------------------------------------------------
Env/Reward/Episode (New)                               |  7.51
Env/Length/Episode (New)                               |  7.51
Env/Reward/Episode (Last 50)                           |  7.38
Env/Length/Episode (Last 50)                           |  7.38
policy/loss                                            |  7.62
v/loss                                                 | 45.51
--------------------------------------------------------------

                         Step 10752                         
--------------------------------------------------------------
Env/Reward/Episode (New)                              |   8.80
Env/Length/Episode (New)                              |   8.80
Env/Reward/Episode (Last 50)                          |  12.40
Env/Length/Episode (Last 50)                          |  12.40
policy/loss                                           |  


                        Step 154112                        
--------------------------------------------------------------
Env/Reward/Episode (New)                             |  187.00
Env/Length/Episode (New)                             |  187.00
Env/Reward/Episode (Last 50)                         |  187.38
Env/Length/Episode (Last 50)                         |  187.38
policy/loss                                          |   44.09
v/loss                                               | 2108.07
--------------------------------------------------------------

                        Step 164352                        
--------------------------------------------------------------
Env/Reward/Episode (New)                             |  238.70
Env/Length/Episode (New)                             |  238.70
Env/Reward/Episode (Last 50)                         |  225.80
Env/Length/Episode (Last 50)                         |  225.80
policy/loss                                          |   76


                        Step 307712                        
--------------------------------------------------------------
Env/Reward/Episode (New)                             |  856.08
Env/Length/Episode (New)                             |  856.08
Env/Reward/Episode (Last 50)                         |  760.28
Env/Length/Episode (Last 50)                         |  760.28
policy/loss                                          |   33.33
v/loss                                               | 1435.71
--------------------------------------------------------------

                        Step 317952                        
--------------------------------------------------------------
Env/Reward/Episode (New)                             |  610.81
Env/Length/Episode (New)                             |  610.81
Env/Reward/Episode (Last 50)                         |  724.98
Env/Length/Episode (Last 50)                         |  724.98
policy/loss                                          |   53


                        Step 461312                        
--------------------------------------------------------------
Env/Reward/Episode (New)                              | 909.08
Env/Length/Episode (New)                              | 909.08
Env/Reward/Episode (Last 50)                          | 919.72
Env/Length/Episode (Last 50)                          | 919.72
policy/loss                                           |  10.72
v/loss                                                | 593.78
--------------------------------------------------------------

                        Step 471552                        
--------------------------------------------------------------
Env/Reward/Episode (New)                              | 863.09
Env/Length/Episode (New)                              | 863.09
Env/Reward/Episode (Last 50)                          | 907.74
Env/Length/Episode (Last 50)                          | 907.74
policy/loss                                           |  29